In [ ]:
library(ggplot2)
library(cluster)

# Clustering

## Comparing Clusters

<div>
<img src="inputs/comparingClusters.png" width=50%/>
</div>

### Centroids

$$c(A) = \frac{1}{|A|}\sum_{x \in A} x$$

Distance between two clusters is:

$$d(C_1, C_2) = d(c(C_1), c(C_2))$$

### UPGMA - unweighted pair group method with arithmetic mean

$$d(C_1, C_2) = \frac{1}{|C_1| \times |C_2|}\sum_{x \in C_1}\sum_{y \in C_2}d(x,y)$$

## Hierarchical Clustering

The algorithm works as follows:

* Put each data point in its own cluster.
* Identify the closest two clusters and combine them into one cluster.
* Repeat the above step till all the data points are in a single cluster.

In [ ]:
levels(iris$Species)

In [ ]:
head(iris[,3:4])

`hclust()` syntax: `hclust(data, method)`, where `data` is a dissimilarity structure

In [ ]:
d = dist(iris[, 3:4], method = "euclidean")
d

In general, for a data sample of size M, the distance matrix is an M × M symmetric matrix with M × (M - 1)∕2 distinct elements. Hence for a data sample of size 150 (the number of observations in `iris`), its distance matrix has about 11,000 distinct elements.

In [ ]:
clusters1 <- hclust(d) #defaults to complete linkage method (not covered)
clusters1 # object of class hclust

In [ ]:
plot(clusters1)
rect.hclust(clusters1 , k = 3, border = 2:6)

In [ ]:
clusterCut1 <- cutree(clusters1, 3)
clusterCut1

In [ ]:
table(clusterCut1, iris$Species)

In [ ]:
ggplot(iris, aes(x=Petal.Length, y=Petal.Width, color=Species)) + geom_point(size = 2)

### Question

Let's see if using a different clustering method will give better results. Try the `average` method on your own (add the argument `method` to the `hclust` function). Plot the respective dendogram. Compare cluster membership among k=3 clusters and the members between species.

## K-means Clustering

In k means clustering, we have to specify the number of clusters, $k$, we want the data to be grouped into. The algorithm randomly picks $k$ observations and assigns them each to their own cluster. These points are the centroids of their clusters. Then, the algorithm iterates through two steps:
* Reassign data points to the cluster whose centroid is closest.
* Calculate new centroid of each cluster.

These two steps are repeated till the within cluster variation cannot be reduced any further. 

`kmeans()` syntax: `kmeans(data, centers)`

In [ ]:
set.seed(20) #ensures everyone gets the same results
kCluster <- kmeans(iris[, 3:4], 3, nstart = 20)
kCluster

In [ ]:
kCluster$tot.withinss

In [ ]:
table(kCluster$cluster, iris$Species)

As we can see, the data belonging to the setosa species got grouped into cluster 3, versicolor into cluster 1, and virginica into cluster 2. The algorithm wrongly classified two data points belonging to versicolor and four data points belonging to virginica.

We can also plot the data to see the clusters:

In [ ]:
kCluster$cluster <- as.factor(kCluster$cluster)
ggplot(iris, aes(Petal.Length, Petal.Width, color = kCluster$cluster)) + geom_point(size = 2)

In [ ]:
ggplot(iris, aes(x=Petal.Length, y=Petal.Width, color=Species)) + geom_point(size = 2)

In [ ]:
ggplot(iris, aes(Petal.Length, Petal.Width, color = Species)) +
    geom_point(alpha = 0.4, size = 3.5) + geom_point(col = kCluster$cluster) +
    scale_color_manual(values = c('green', 'black', 'red'))


### Determining optimal number of clusters (k)

#### Elbow Plot

$$TWCSS = \sum_{i=1}^k\sum_{x \in C_i}(x-\mu_i)^2$$
$$\mu_i = c(C_i)$$

In [ ]:
#Elbow Method for finding the optimal number of clusters
# Compute and plot wss for k = 1 to k = 7.
k.max <- 7 #max 150 (num of obs)
twcss <- sapply(1:k.max, 
              function(k){kmeans(iris[, 3:4], k, nstart=20)$tot.withinss})

plot(1:k.max, twcss,
     type="b", pch = 19, frame = FALSE, 
     xlab="Number of clusters K",
     ylab="Total within-clusters sum of squares")


#### Silhouette Score

$$a(x) = \frac{1}{|C_i|-1}\sum_{y \in C_i, x \neq y}d(x,y)$$
$$b(x) = min_{k \neq i}\big(\frac{1}{|C_i|}\sum_{y \in C_k}d(x,y)\big)$$
$$s(x) = \begin{cases} \frac{b(x)-a(x)}{max(a(x),b(x))}, & \text{if $|C_i| > 1$}.\\ 0, & \text{if $|C_i| = 1$}. \end{cases}$$

The silhouette *score* is the $mean(s(x))$ for all $x$.

In [ ]:
k=3
km <- kmeans(iris[, 3:4], centers = k, nstart=20)
ss <- silhouette(km$cluster, dist(iris[, 3:4], "euclidean"))
ss

In [ ]:
#silhouette method
avg_silhouette_score <- function(k){
  km <- kmeans(iris[, 3:4], centers = k, nstart=20)
  ss <- silhouette(km$cluster, dist(iris[, 3:4], "euclidean"))
  mean(ss[, 3])
}
k <- 2:7 #minimum number of clusters for silhouette scores is 2 for between-cluster variation
avg_sil <- sapply(k, avg_silhouette_score)
k <- c(0, k)
avg_sil <- c(0, avg_sil)
plot(k, avg_sil, type='b', , xlab='Number of clusters', ylab='Average Silhouette Scores', frame=FALSE, ylim = c(0,0.8))

### Question

Run a K-means analysis for 2 clusters. Compare the table for membership within clusters and membership within species for this analysis to the table from the k=3 analysis. Do you notice anything interesting? 

Food for thought: what biological reason might there be that versicolor and virginica are getting clustered together?

## Assignment

In [ ]:
library(ggplot2)
library(cluster)

1. Read in the hw12.csv and make a new Data.Frame out of the columns Annual Income & Spending Score (1-100).

    (A) Perform centroid hierarchical clustering on the dataset and visualize it in a dendogram.

    (B) Perform UPGMA hierarchical clustering on the dataset and visualize it in a dendogram.

2. Choose one of the above methods. Cut the dendogram using cutree into k=2 clusters. Create a table that compares the clustering to the gender of each customer (refer back to the original data). Is there reason to believe the data is dependent on gender? Why or why not?

3. Perform a K-means analysis, from 1 to 10 clusters. Create an elbow plot. Which value does the plot suggest is the optimal number of clusters?

4. Perform a K-means analysis, up to 10 clusters. Create a silhouette plot (recall the minimum number of clusters needed for this kind of plot). Does this value agree with the value from question 3?